In [8]:
import sys
sys.path.append("../scripts")

In [52]:
from helper_funcs import *	
from multiprocessing import Pool, cpu_count

from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)


def check_file(f):
    exists = os.path.isfile(f)
    if not exists:
        msg = 'missing'
    else:
        with suppress_stdout_stderr():
            try:
                res =cdo.sinfov(input=f)
                msg = "OK"
            except Exception:
                msg = "corrupted"
    
    return msg



In [74]:
del year, model, variable, scen

In [50]:

num = {k:len(v) for k, v in possibilities.items()}

In [105]:
def expected_num_files(m, v, scen):
    if scen == "historical":
        n_h = 1
        n_f = 0
    elif scen == "future":
        n_h = 0
        n_f = 1
    elif scen == "both":
        n_h = 1
        n_f = 1
    elif scen == "all":
        n_h = 1
        n_f = 2
        
    exp_val = m*v*n_h*(2005-1950 +1) + m*v*n_f*(2099-2006 +1)
    
    return(exp_val)

In [108]:
expected_num_files(21, 3, "all")

15372

In [95]:
class DummyFile:
    def __init__(self, status):
        self.status = status
    
    def get(self):
        return self.status
    
    
DummyFile("missing").get()

'missing'

In [116]:
pool = Pool(7)

files_status = [None]*expected_num_files(2, 3, "all")

counter=0
for model in possibilities['model'][2:4]:
    for variable in possibilities['variable']:
        for scen in possibilities['scen']:
            if scen == "historical":
                years = range(1950, 2005 + 1)
            else:
                years = range(2006, 2099 + 1)

            for year in years:
                args = [year, model, variable, scen]
                try:
                    f = get_model_year_file(*args)[0]
                    status = pool.apply_async(check_file, [f])
                except IndexError:
                    status = DummyFile(status="missing")
                    
                files_status[counter] = (args + [status])
                
                counter += 1

pool.close()
pool.join()

In [112]:
files_status

[[1950,
  'BNU-ESM',
  'pr',
  'historical',
 [1951,
  'BNU-ESM',
  'pr',
  'historical',
 [1952,
  'BNU-ESM',
  'pr',
  'historical',
 [1953,
  'BNU-ESM',
  'pr',
  'historical',
 [1954,
  'BNU-ESM',
  'pr',
  'historical',
 [1955,
  'BNU-ESM',
  'pr',
  'historical',
 [1956,
  'BNU-ESM',
  'pr',
  'historical',
 [1957,
  'BNU-ESM',
  'pr',
  'historical',
 [1958,
  'BNU-ESM',
  'pr',
  'historical',
 [1959,
  'BNU-ESM',
  'pr',
  'historical',
 [1960,
  'BNU-ESM',
  'pr',
  'historical',
 [1961,
  'BNU-ESM',
  'pr',
  'historical',
 [1962,
  'BNU-ESM',
  'pr',
  'historical',
 [1963,
  'BNU-ESM',
  'pr',
  'historical',
 [1964,
  'BNU-ESM',
  'pr',
  'historical',
 [1965,
  'BNU-ESM',
  'pr',
  'historical',
 [1966,
  'BNU-ESM',
  'pr',
  'historical',
 [1967,
  'BNU-ESM',
  'pr',
  'historical',
 [1968,
  'BNU-ESM',
  'pr',
  'historical',
 [1969,
  'BNU-ESM',
  'pr',
  'historical',
 [1970,
  'BNU-ESM',
  'pr',
  'historical',
 [1971,
  'BNU-ESM',
  'pr',
  'historical',
 [1972,
  

In [21]:
import pandas as pd

In [117]:
df = pd.DataFrame(files_status, columns=['year', 'model', 'variable', 'scen', 'status'])

In [118]:
%time
df.status = df.status.apply(lambda x: x.get())

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs


In [128]:
df = pd.read_csv("../../pickles/check-model-files/files_status.csv")

In [154]:
dont_merge = (df
              .loc[df.status != "OK"]
              .groupby(['model', 'variable', 'scen'])
              .size().reset_index().drop(0, axis=1))

In [159]:
outdir = '../../pickles/check_file_status'
os.system(f'mkdir -p {outdir}')
dont_merge.to_csv(f'{outdir}/dont_merge.csv', index=False)